<a href="https://colab.research.google.com/github/sampath9dasari/adversarial-machine-learning/blob/master/Homomorphic_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Privacy Preserving Machine Learning

First things first. Let's run the package installations. They take quite a while. So hit run on the cell below before continuing with this introduction.


Executing? Perfect!  

Consider the following scenario: You are business that speaclizes in machine learning. You have trained some great models on data that has been carefully collected and labeled. The data is quite sensitve and you had to jump through a lot of legal and hoops to get access to it. In this notebook this data will be represented by the android permission data that we have been working so far. Since that you are working on is sensitve and hard to get you are faced with a porblem. Your clients are reclutant to give you their data but at the same time you don't want to give your model to them either. 
But there are solutions to this problem and it this notebook we will explore to of those. Namely Secure Multiparty Computation SMC (also often called just Multi Party Computation  MPC) and Homomorphic Encryption(HE). Both are cryptographic ways of performing computation on data that is being kept secret. Here we will be focusing on HE



In [1]:
!pip install Pyfhel

Next we'll get our usual boilerplat code out of the way. Data loading, splitting, etc.

In [2]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# select a subset of the data
# we only wants ones and zeros
# 200 instances per class

# instances
x_train = np.concatenate( [ x_train[ y_train == 0 ][ :200 ], x_train[ y_train == 1 ][ :200 ] ] )
x_test = np.concatenate( [ x_test[ y_test == 0 ][ :200 ], x_test[ y_test == 1 ][ :200 ] ] )
# x_train = x_train.astype( float ) / 255.
# x_test = x_test.astype( float ) / 255.


x_train_rounded = np.round( x_train )

print( 'training data: ', x_train.shape )
print( 'test data: ', x_test.shape )

# labels
y_train = np.concatenate( [ np.zeros( 200 ), np.ones( 200 ) ] )
y_test = np.concatenate( [ np.zeros( 200 ), np.ones( 200 ) ] )

print( 'training data: ', y_train.shape )
print( 'test data: ', y_test.shape )


training data:  (400, 28, 28)
test data:  (400, 28, 28)
training data:  (400,)
test data:  (400,)


## Fully Homomorphic encryption

Fully Homomorphic encryption is a tool that can be used for PPML. It does not rely on splitting the secret between parties to jointly evaluate a function. It is more like "traditional" cryptography in the sense that the one party encrypts the data. Any other party can perform computation the data without the need for decrypting it. The result of the computation is still encrypted. 

Opposed to whwat we have been doing so far we will not be working with a high level library but rather will build our own functions on top of simple operations.



In [0]:
from Pyfhel import Pyfhel, PyPtxt, PyCtxt
import time

# Pyfhel class contains most of the functions.
# PyPtxt is the plaintext class
# PyCtxt is the ciphertext class


HE = Pyfhel()           
# p (long): Plaintext modulus. All operations are modulo p.
# m (long=2048): Coefficient modulus.
# flagBatching (bool=false): Set to true to enable batching.
# base (long=2): Polynomial base.
# sec (long=128): Security level equivalent in AES. 128 or 192.
# intDigits (int=64): truncated positions for integer part.
# fracDigits (int=32): truncated positions for fractional part.
HE.contextGen(p=65537)  

# generate keys
HE.keyGen()           



Before we can encrypt nmumber we need to encode them. After that we can perform computation on the ciphertexts. Once we decrypt the result we need to decode it into the desired format.

In [4]:
# plaintext values
a = 1
b = 2

# encode
a = HE.encodeInt( a )
print('a:', a.to_string() )
b = HE.encodeInt( b )
print('b:', b.to_string() )

# encrypt
a_ctxt = HE.encrypt( a )
b_ctxt = HE.encrypt( b )

# perform computation
result = a_ctxt + b_ctxt
decrypted = HE.decrypt( result )

# decrypt
print( 'decrypted:', decrypted.to_string() ) 

# decdode
print( 'decoded:', HE.decodeInt( decrypted ) )



a: b'1'
b: b'1x^1'
decrypted: b'1x^1 + 1'
decoded: 3


Thankfully we don't have to encode and decode evertime. There are convience methods for it.

In [5]:
# plaintext values
a = 1
b = 2

# encpde and encrypt
a_ctxt = HE.encryptInt( a )
b_ctxt = HE.encryptInt( b )

# perform computation
result = a_ctxt + b_ctxt

# decrypt and decdode
print( 'decerypted and decoded:', HE.decryptInt( result ) )

decerypted and decoded: 3


Using the functions `encodeFrac`, `decodeFrac` and `encryptFrac` and `decryptFrac` to replicate the firs example with float values. What do you notice about the encoding?

In [6]:
# plaintext values
a = .1
b = .2

# encode
a = HE.encodeFrac( a )
print('a:', a.to_string() )
b = HE.encodeFrac( b )
print('b:', b.to_string() )

# encrypt
a_ctxt = HE.encrypt(a)
b_ctxt = HE.encrypt(b)

# perform computation
result = a_ctxt + b_ctxt
decrypted = HE.decrypt( result )

# decrypt
print( 'decrypted:', decrypted.to_string() ) 

# decdode
print( 'decoded:',  HE.decodeFrac( decrypted ))

a: b'10000x^2044 + 10000x^2043 + 10000x^2040 + 10000x^2039 + 10000x^2036 + 10000x^2035 + 10000x^2032 + 10000x^2031 + 10000x^2028 + 10000x^2027 + 10000x^2024 + 10000x^2023 + 10000x^2020 + 10000x^2019 + 10000x^2016'
b: b'10000x^2045 + 10000x^2044 + 10000x^2041 + 10000x^2040 + 10000x^2037 + 10000x^2036 + 10000x^2033 + 10000x^2032 + 10000x^2029 + 10000x^2028 + 10000x^2025 + 10000x^2024 + 10000x^2021 + 10000x^2020 + 10000x^2017 + 10000x^2016'
decrypted: b'10000x^2045 + FFFFx^2044 + 10000x^2043 + 10000x^2041 + FFFFx^2040 + 10000x^2039 + 10000x^2037 + FFFFx^2036 + 10000x^2035 + 10000x^2033 + FFFFx^2032 + 10000x^2031 + 10000x^2029 + FFFFx^2028 + 10000x^2027 + 10000x^2025 + FFFFx^2024 + 10000x^2023 + 10000x^2021 + FFFFx^2020 + 10000x^2019 + 10000x^2017 + FFFFx^2016'
decoded: 0.2999999998137355


But what about the noise? I thought there was noise involved int HE?

In [7]:
HE = Pyfhel()           
HE.contextGen( p=65537 )  
# generate keys
HE.keyGen()      

# plaintext values
a = 1
b = 2

# encpde and encrypt
a_ctxt = HE.encryptInt( a )
b_ctxt = HE.encryptInt( b )

# perform computation
result = a_ctxt * b_ctxt
result = result * a_ctxt

print( 'decerypted and decoded:', HE.decryptInt( result ) )


decerypted and decoded: -4734767386158461553


In [8]:
HE = Pyfhel()           
# p (long): Plaintext modulus. All operations are modulo p.
# m (long=2048): Coefficient modulus.
# flagBatching (bool=false): Set to true to enable batching.
# base (long=2): Polynomial base.
# sec (long=128): Security level equivalent in AES. 128 or 192.
# intDigits (int=64): truncated positions for integer part.
# fracDigits (int=32): truncated positions for fractional part.
HE.contextGen( p=65537, m=4096 )  

# generate keys
HE.keyGen()      

# plaintext values
a = 1
b = 2

# encpde and encrypt
a_ctxt = HE.encryptInt( a )
b_ctxt = HE.encryptInt( b )

# perform computation
result = a_ctxt * b_ctxt
result = result * a_ctxt

print( 'decerypted and decoded:', HE.decryptInt( result ) )

decerypted and decoded: 2


For a simple example consider the following scenario. We are still working with the MNIST data set (that we all know and love) but to keep things simple we are only using two classes and small amount of data. First we are training a simple classifier on plain data. Namely a perceptron. 

In [9]:
from sklearn.linear_model import Perceptron

percp = Perceptron(fit_intercept=False)
percp.fit( x_train.reshape( ( x_train.shape[ 0 ], -1 ) ), y_train )
print( 'test score: ', percp.score( x_test.reshape( ( x_train.shape[ 0 ], -1 ) ), y_test ) )


print( 'prediction:', percp.predict( x_test[ 1:2 ].reshape( ( 1, -1 ) ) ) )
print( 'output:', percp.decision_function( x_test[ 1:2 ].reshape( ( 1, -1 ) ) ) )
print( 'actual lable:', y_test[ 1:2 ] )

test score:  1.0
prediction: [0.]
output: [-3543691.]
actual lable: [0.]


In [10]:
percp.coef_[0].shape

(784,)

Let's transfere the the perceptron algorithm to the encrypted domain. We can perform operations between plaintexts and ciphertexts but we need to encode the plaintexts first

In [15]:
from Pyfhel import Pyfhel, PyPtxt, PyCtxt
import time

# Pyfhel class contains most of the functions.
# PyPtxt is the plaintext class
# PyCtxt is the ciphertext class


HE = Pyfhel()           
# p (long): Plaintext modulus. All operations are modulo p.
# m (long=2048): Coefficient modulus.
# flagBatching (bool=false): Set to true to enable batching.
# base (long=2): Polynomial base.
# sec (long=128): Security level equivalent in AES. 128 or 192.
# intDigits (int=64): truncated positions for integer part.
# fracDigits (int=32): truncated positions for fractional part.
HE.contextGen(p=65537, m=4096)  

# generate keys
HE.keyGen()           

# encrypt values
x_ctxt = [HE.encrypt(x) for x in x_test[1].reshape(-1)]
prediction = HE.encrypt(0)

# encode weights
weights = [HE.encryptInt(x) for x in percp.coef_[0].reshape(-1)]

start = time.time()

# perform prediction
for w, x in zip(weights, x_ctxt):
  temp = w*x
  prediction = prediction + temp

# decrypt results
print( 'prediction took:', time.time() - start )
result = HE.decryptInt( prediction )
print( 'prediction:', result )
print( 'actual label:', y_test[ 1 ] )


prediction took: 5.019931793212891
prediction: -3543691
actual label: 0.0


let's do it with SIMD

In [38]:
HE = Pyfhel()           
HE.contextGen( p=65537, flagBatching=True, )   

# generate keys
HE.keyGen()    

# plain data
a = [ 1,2,3,4 ]
b = 2

a = HE.encodeBatch( a )
print( 'encoded:', a.to_string() )

a = HE.encrypt( a )

# adding another value
try:
  print( 'try 1')
  b_enc = HE.encodeInt( b )
  a = a + b_enc
  print( 'success!!')
except Exception as e:
  print( e )

try:
  print( 'try 2')
  b_enc = HE.encodeBatch( b )
  a = a + b_enc
  print( 'success!!')
except Exception as e:
  print( e )

try:
  print( 'try 3')
  b_enc = HE.encodeBatch( [b] * 4 )
  a = a + b_enc
  print( 'success!!')
except Exception as e:
  print( e )

print( 'decoded and decrypted: ', HE.decryptBatch( a ) )


encoded: b'2E20x^2047 + 69FBx^2046 + A71Ax^2045 + 93E3x^2044 + 8D57x^2043 + AF8Ax^2042 + AD31x^2041 + 7E99x^2040 + 5F6Dx^2039 + AC6x^2038 + AC38x^2037 + 3A71x^2036 + 7EF1x^2035 + 86D2x^2034 + 7C24x^2033 + C543x^2032 + 3548x^2031 + 51CAx^2030 + 4DA0x^2029 + 548Ex^2028 + E4F0x^2027 + 5419x^2026 + 6E48x^2025 + 911Ex^2024 + 4D17x^2023 + C36Ax^2022 + EA40x^2021 + A04Cx^2020 + AD3Dx^2019 + C605x^2018 + CE37x^2017 + 6BC9x^2016 + 2D6Ax^2015 + 2A4Dx^2014 + 4798x^2013 + D1E6x^2012 + D72Fx^2011 + 7B7Cx^2010 + CA8Dx^2009 + FF75x^2008 + E389x^2007 + EF2x^2006 + 2D29x^2005 + 5E22x^2004 + 1BD3x^2003 + 843Dx^2002 + F779x^2001 + C66Cx^2000 + 5E74x^1999 + 6DAEx^1998 + 9949x^1997 + 2301x^1996 + 517Ex^1995 + 8DC0x^1994 + BD10x^1993 + 8B41x^1992 + 7FCCx^1991 + 6378x^1990 + 25E8x^1989 + 96DBx^1988 + C265x^1987 + 41Cx^1986 + BBA7x^1985 + BB74x^1984 + 2AFAx^1983 + CF09x^1982 + A6x^1981 + 91D7x^1980 + 4657x^1979 + 4699x^1978 + C8A8x^1977 + D0B5x^1976 + D155x^1975 + 9CECx^1974 + 4F7Cx^1973 + 64EDx^1972 + E85Ex^

In [0]:
HE = Pyfhel()           
HE.contextGen( p=65537, flagBatching=True )  
HE.keyGen()   
# need to get data into the correct shape
x_test = x_test.reshape( (x_test.shape[ 0 ], -1 ) )

slots = HE.getnSlots()
num_features = x_test.shape[ 1 ]

print( x_test.shape )

# encrypt values
# iterate over every feature
cipher_texts = []


# encode weights


start = time.time()

# perform prediction

# decrypt results
print( 'prediction took:', time.time() - start )
result = HE.decryptBatch( prediction )
print( result )
print( len(result) )


print( percp.decision_function( x_test ) )


Why did that not work?

In [0]:
# change the weights to be smaller

# create a copy of the preceptron
percp1 = Perceptron(fit_intercept=False)
percp1.classes_ = percp.classes_ 
coef = np.copy( percp.coef_ )

# make changes to the coefficents

percp1.coef_ = coef
percp1.intercept_ = percp.intercept_

print( 'test score: ', percp1.score( x_test.reshape( ( x_train.shape[ 0 ], -1 ) ), y_test ) )

In [0]:
HE = Pyfhel()           
HE.contextGen( p=65537, flagBatching=True )  
HE.keyGen()   
# need to get data into the correct shape
x_test = x_test.reshape( (x_test.shape[ 0 ], -1 ) )

slots = HE.getnSlots()
num_features = x_test.shape[ 1 ]

print( x_test.shape )

# encrypt values
# iterate over every feature
cipher_texts = []


# encode weights


start = time.time()

# perform prediction

# decrypt results
print( 'prediction took:', time.time() - start )
result = HE.decryptBatch( prediction )
print( result )
print( len(result) )


print( percp1.decision_function( x_test ) )

putting together the building blocks

In [0]:
x_train = x_train.reshape( ( x_train.shape[ 0 ], -1 ) )
x_test = x_test.reshape( ( x_test.shape[ 0 ], -1 ) )

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical



print( x_train.shape )

model = Sequential()
model.add( Dense( 2, activation='relu', input_shape=x_train.shape[ 1: ]  ) )
model.add( Dense( 1, activation='sigmoid' ) )


model.summary()
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.fit( x_train, y_train, epochs=32, verbose=1 )
model.evaluate( x_test, y_test )

lets build a model that can work with HE

In [0]:
def relu_aprox( x ):
  return 0.046875*x**2 + 0.5*x + 0.9375

def sig_aprox( x ):
   return 0.424413181578472*x**2 + 0.500000000000002*x + 0.106103295394586

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.


model = Sequential()
model.add( Dense( 2 , activation=relu_aprox, input_shape=x_train.shape[ 1: ]  ) )
model.add( Dense( 1, activation='sigmoid' ) )

model.summary()
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.fit( x_train, y_train, epochs=32 )

print( 'keras' )
print( model.evaluate( x_test, y_test ) )
print( 'prediction' )
test_sample = x_test[ 0:1 ]
print( 'prediction', model.predict( test_sample ) )




Extract weights and setup the encryption scheme


In [0]:
#-------------------------------------------------------------------------------
# setup HE
#-------------------------------------------------------------------------------
print('HE')
HE = Pyfhel()           
HE.contextGen(p=65537, m=4096)  
# generate keys
HE.keyGen()    

# extract weights
print( 'weights layer 0' )
layer0_weights = model.layers[ 0 ].get_weights() # format [ weights, biases ]
print( layer0_weights[ 0 ].shape, layer0_weights[ 1 ].shape )

print( 'weights layer 1' )
layer1_weights = model.layers[ 1 ].get_weights()
print( layer1_weights[ 0 ].shape, layer1_weights[ 1 ].shape )



# let's implement the actual layers
#-------------------------------------------------------------------------------
# layers
#-------------------------------------------------------------------------------



Convert values and encrypt

In [0]:
#-------------------------------------------------------------------------------
# convert values
#-------------------------------------------------------------------------------

def weight_converter( weights, biases ):
  bias_ = []     # holds converted biases 
  weights_ = []  # holds converted weights 

  # convert biases 
  for b in biases:
    bias_.append( HE.encodeFrac( b ) )

  # convert weights
  for input in weights:
    w = []
    for weight in input:
      w.append( HE.encodeFrac( weight ) )
      weights_.append( w )  
  
  return weights_, bias_

# layer 0
weights_0, bias_0 = weight_converter( layer0_weights[ 0 ], layer0_weights[ 1 ] )
# layer 1
weights_1, bias_1 = weight_converter( layer1_weights[ 0 ], layer1_weights[ 1 ] )


# convert values for activation functions
relu_aprox_coef = [ HE.encodeFrac( 0.046875 ),  HE.encodeFrac( 0.5 ), 
              HE.encodeFrac( 0.9375 ) ]

sig_aprox_coef = [ HE.encodeFrac( 0.424413181578472 ),
                  HE.encodeFrac( 0.500000000000002 ),
                  HE.encodeFrac( 0.106103295394586 ) ]

#-------------------------------------------------------------------------------
# encrypt inputs
#-------------------------------------------------------------------------------
inputs = [ HE.encryptFrac( x ) for x in test_sample[ 0 ] ]

Now it is your turn. Implement the layers. Good Luck :D